Most existing approaches to visual odometry are based on the following stages.

1. Acquire input images
2. Image correction: apply image processing techniques for lens distortion removal, etc.
3. Feature detection: define interest operators, and match features across frames and construct optical flow field.
       
4. Check flow field vectors for potential tracking errors and remove outliers.
5. Estimation of the camera motion from the optical flow.
       
6. Periodic repopulation of trackpoints to maintain coverage across the image.

In [4]:
################################Testing################################

In [1]:
################################Oxford Dataset################################
%run robotcar-sdk/python/camera_model.py
from skimage.io import imread_collection

#oxford(ox) setup
models_dir = 'robotcar-sdk/models'
ox_img_dir = 'oxford_2014-05-14-13-50-20/stereo/left/'
ox_cam = CameraModel(models_dir,ox_img_dir)
ox_imgs = imread_collection(ox_img_dir+'*.png', conserve_memory = True)
# ox_imgs = np.array(ox_imgs)
print "ox loaded."

ox loaded.


In [2]:
################################KITTI Dataset################################
#adapted from https://github.com/uoip/monoVO-python 
%run pinhole_camera.py

##kitti setup
poses_dir = 'dataset/poses/00.txt' #for ground truth
img_dir = 'dataset/sequences/00/image_0/'
cam = PinholeCamera(1241.0, 376.0, 718.8560, 718.8560, 607.1928, 185.2157)
print "kitti loaded."

kitti loaded.


In [ ]:
vo = VisualOdometry(cam, poses_dir)

# ox_vo = VisualOdometry(ox_cam, poses_dir)

traj = np.zeros((600,600,3), dtype=np.uint8)

#drawing trajectories for each image
for img_id in xrange(10000):
    img = cv2.imread(img_dir+str(img_id).zfill(6)+'.png', 0)
    
    vo.update(img, img_id)

    cur_t = vo.cur_t
    if(img_id > 2): x, y, z = cur_t[0], cur_t[1], cur_t[2]
    else: x, y, z = 0., 0., 0.
        
    #offset so the 2 trajectories do not overlap
    x_offset, y_offset = 0, 0
    draw_x, draw_y = int(x)+(290-x_offset), int(z)+(90-y_offset)
    true_x, true_y = int(vo.trueX)+290, int(vo.trueZ)+90

    #predicted trajectory
    cv2.circle(traj, (draw_x,draw_y), 1, (img_id*255/4540,255-img_id*255/4540,0), 1)
    #actual trajectory
    cv2.circle(traj, (true_x,true_y), 1, (0,0,255), 1)
    
    cv2.rectangle(traj, (10, 20), (600, 60), (0,0,0), -1)
    text = "Coordinates: x=%2fm y=%2fm z=%2fm"%(x,y,z)
    cv2.putText(traj, text, (20,40), cv2.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1, 8)

    cv2.imshow('Road facing camera', img)
    cv2.imshow('Trajectory', traj)
    cv2.waitKey(1)

cv2.imwrite('map.png', traj)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
##########################Adapting Existing Code################################

In [ ]:
%run pinhole_camera.py
import cv2
#adapted from https://github.com/avisingh599/mono-vo
def feautre_tracking(img1, img2, points1, points2, status):
    errs = []
    #lucas kanade(lk) optical flow parameters
    lk_params = dict(winSize  = (21, 21), 
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))
    
    #open cv method for generating LK optical flow
    #shape: [k,2] [k,1] [k,1]
    cv2.calcOpticalFlowPyrLK(img1, img2, points1, points2, status, errs, **lk_params)
    
    #removing points for which the tracking has failed/gone outside the frame
    indexCorrection = 0
    for i in range(len(status)):
        pt = points2[i-indexCorrection]
        if(status[0] == 0 or (pt[0]<0 or pt[1]<0)):
            if(pt[0]<0 or pt[1]<0):
                status[i]=0
            del points1[i-indexCorrection]
            del points2[i-indexCorrection]           
            indexCorrection+=1
    
#      calcOpticalFlowPyrLK(img_1, img_2, points1, points2, status, err, winSize, 3, termcrit, 0, 0.001);

def feature_detection(img1, points1):
    key_points = []
    threshold_value = 20
    nonMax = True
    key_points = cv2.FastFeatureDetector_create(threshold=threshold_value, nonmaxSuppression=nonMax)
    
##kitti setup
poses_dir = 'dataset/poses/00.txt' #for ground truth
img_dir = 'dataset/sequences/00/image_0/'
cam = PinholeCamera(1241.0, 376.0, 718.8560, 718.8560, 607.1928, 185.2157)